In [36]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

In [37]:
# Image path
imagePath = 'Images'
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

mountingPoints = []
quadPoints = []

Steps to do test/implement ray masking
1. Plot out viewing quadrilateral
2. Plot out mounting point
3. Mask out for building contours
4. Find corners of building contours

Taking the nearest corner, find the slope of the line joining the mounting point and this nearest corner. Find the intersection point of this line and the farthest edge of the quadrilateral (What if farthest edge is out of bounds ??)

In [38]:
def sameSideCheck(pt_A,pt_B,pt_C):
    #use dot product of AB and AC to checked if C and B are on the same side of the pt A
    vec_AB_x = pt_B[0] - pt_A[0]
    vec_AB_y = pt_B[1] - pt_A[1]
    mag_AB = math.sqrt(pow(vec_AB_x,2) + pow(vec_AB_y,2))

    vec_AC_x = pt_C[0] - pt_A[0]
    vec_AC_y = pt_C[1] - pt_A[1]
    mag_AC = math.sqrt(pow(vec_AC_x,2) + pow(vec_AC_y,2))

    check = (vec_AB_x*vec_AC_x + vec_AB_y*vec_AC_y)/(mag_AB*mag_AC)

    if(check>0):
        return True
    return False

In [39]:
def getBorderContour_text(img):
#function runs only once
#using new bgr values for the new image in low red.

    img_c = img.copy()
    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    red_mask = cv2.inRange(img, low_red, low_red )
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape, dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.01, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, (255,255,255), thickness=1)
            # cv2.drawContours(blank, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank,masked

In [40]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        mountingPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 0, 0), 2)
        cv2.imshow('img', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        quadPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y),(x,y), font, 1,(0, 0, 255), 2)
        cv2.imshow('img', img)

In [41]:
def getCorners(buildingBorderMask, quadPts, blocksize=3):

    # ~~~~~~~~~~~~~ #
    # for obtaining corners for ray masking
    # ~~~~~~~~~~~~~ #

    # Mask may have 3 layers
    operatedImage = cv2.cvtColor(buildingBorderMask, cv2.COLOR_BGR2GRAY)

    # uncomment if you feel image is noisy (not needed)
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) 

    # modify the data type -- setting to 32-bit floating point
    operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method to detect the corners with appropriate values as input parameters
    # increase the second parameter ~ blocksize to get more of the corner shape out
    dest = cv2.cornerHarris(operatedImage, blocksize, 5, 0.07)  

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # a mask of all corners of the building contours
    cornerMask = np.zeros(buildingBorderMask.shape, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    # Make a gray scale mask of quadilateral area
    quadMask = np.zeros(buildingBorderMask.shape[:2], dtype='uint8')
    polyPts = np.array( quadPts ,dtype=np.int32)
    polyPts = polyPts.reshape((-1, 1, 2))
    quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

    # a mask of all corners in quadilateral only
    cornerMask = cv2.bitwise_and(cornerMask,cornerMask,mask=quadMask)

    return cornerMask


In [42]:
def findClusterCenters(img, excludedCornerPoints = []):

    corner_centers = []

    onePixCornerMask = np.zeros(img.shape, dtype='uint8')
    # blank_copy = blank.copy()

    # create masks
    white_mask = cv2.inRange(img, (255,255,255), (255,255,255))

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        if([cx,cy] not in excludedCornerPoints):
            onePixCornerMask[cy, cx] = [255, 255, 255]
            # cv2.circle(onePixCornerMask, (cx, cy), 1, (255, 255, 255), -1)
            corner_centers.append([cx,cy])

    onePixCornerMask = cv2.cvtColor(onePixCornerMask, cv2.COLOR_BGR2GRAY)

    return corner_centers, onePixCornerMask


In [43]:
def getCorners_w_exclude(img, cornerPoints=[]):
    
    operatedImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # modify the data type
    # setting to 32-bit floating point
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate
    # values as input parameters
    #~~~~~~~~~~~~~#
    #for obtaining corners for ray masking
    #~~~~~~~~~~~~~#
    dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # for pts in poly_pts:
    #     dest[pts[0]][pts[1]] = 0
    # Reverting back to the original image,
    # with optimal threshold value
    size = img.shape

    cornerMask = np.zeros(size, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    points,cornerMask = findClusterCenters(cornerMask,cornerPoints)

    return cornerMask, points

In [44]:
img = cv2.imread(imagePath)
cv2.imshow("img",img)

# right click = view quadilateral (4+)
# left click = mounting point (1)
cv2.setMouseCallback('img', click_MountingnScale_points)

# points are to be selected in the order - 
# click 1 = nearest edge left point = quadPoints[0]
# click 2 = nearest edge right point = quadPoints[1]
# click 3 = further edge left point = quadPoints[2]
# click 4 = further edge right point = quadPoints[3]
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
img = cv2.imread(imagePath)

# building contours on black image and building mask
bldg_brdrs, bldg_mask = getBorderContour_text(img)


# cv2.imshow("bldg_brdrs",bldg_brdrs)
# cv2.imshow("bldg_mask",bldg_mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [46]:
# get all the corners in the viewing quadilateral
highlighted_corners = getCorners(bldg_brdrs, quadPoints)
cornerPoints,point_corners = findClusterCenters(highlighted_corners)

# And the corner cluster with building borders to get corner Ls
cornerLs = cv2.bitwise_and(bldg_brdrs,highlighted_corners)
cornerLs = cv2.cvtColor(cornerLs, cv2.COLOR_BGR2GRAY)

# List of all the indices of corner Ls 
cnts=cv2.findContours(cornerLs, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

smallCorners = getCorners(bldg_brdrs, quadPoints, blocksize=2)
_, smallCorners = findClusterCenters(smallCorners)

test = cv2.bitwise_and(cornerLs, cornerLs, mask=smallCorners)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# THIS MIGHT BE WRONG, CHECK LATER
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# inside cnts is a list of contours. Consider one of these lists to be c
# Each contour (c) has any number of points that define it. Due to the nature of the extraction of points, we can assume that
# the corner point is the middlemost point of the point array
# for c in cnts:
#     print(c.tolist())
#     print(len(c))

# copy of the image
img_c = img.copy()

for c in cnts:

    # corresponds to the middlemost index
    cornerIndex = len(c)//2 

    # Draw a circle for debug
    cv2.circle(img_c, (c[cornerIndex,0,0], 
                    c[cornerIndex,0,1]), 1, (255,255,255), 2)


cv2.imshow('small corner', smallCorners)
cv2.imshow('anded', test)
cv2.waitKey(0)
cv2.destroyAllWindows()

# slope = (c[4,0,1] - c[2,0,1])/(c[4,0,0] - c[2,0,0]) 
# print(slope)
# slope = (c[4,0,1] - c[0,0,1])/(c[4,0,0] - c[0,0,0]) 
# print(slope)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [ ]:
#Doing further calculation based on cnts[0], but in main code loop over all contours

c = cnts[0]

cornerIndex = len(c)//2 #corresponds to the middlemost index

#eqs of side edges of quadrilateral
slope_left_edge = (quadPoints[2][1] - quadPoints[0][1])/(quadPoints[2][0] - quadPoints[0][0])
slope_right_edge = (quadPoints[3][1] - quadPoints[1][1])/(quadPoints[3][0] - quadPoints[1][0])
c_left_edge = quadPoints[2][1] - slope_left_edge*quadPoints[2][0]
c_right_edge = quadPoints[1][1] - slope_left_edge*quadPoints[1][0]


#2 points used for generation of equation of line are index 0 and index (-1) [start and end indices]

#eqs of 2 lines from corner point contour (y = mx + c)
#check for divide by zero error
slope_corner_edge_1 = (c[-1,0,1] - c[cornerIndex,0,1])/(c[-1,0,0] - c[cornerIndex,0,0]) 
slope_corner_edge_2 = (c[cornerIndex,0,1] - c[0,0,1])/(c[cornerIndex,0,0] - c[0,0,0]) 
c_corner_edge_1 = c[cornerIndex,0,1] - (slope_corner_edge_1 * c[cornerIndex,0,0])
c_corner_edge_2 = c[cornerIndex,0,1] - (slope_corner_edge_2 * c[cornerIndex,0,0])

isLeftUsed = True   #boolean variable to indicare whether left edge or right edge of viewing quadrilateral is used

#in the following lines of code, we will use the following convention
#for intersection point of contour 1 (points associated with contour 1 are corner point and index -1), point is named - cntr_1_intersection
#for intersection point of contour 2 (points associated with contour 2 are corner point and index 0), point is named - cntr_2_intersection

#find intersection points with both lines
#first find intersection of the line with the left corner edge
cntr_1_intersection_x = (c_corner_edge_1 - c_left_edge)/(slope_left_edge - slope_corner_edge_1)
cntr_1_intersection_y = slope_left_edge*cntr_1_intersection_x + c_left_edge

#check if left corner edge satisfies the condition, if not then assign
if(not(sameSideCheck([c[cornerIndex,0,0],c[cornerIndex,0,1]],[c[-1,0,0],c[-1,0,1]],[cntr_1_intersection_x,cntr_1_intersection_y]))):  #use dot product to check if (x_left_corner_edge_1, y_left_corner_edge_1) is the right intersection point
    cntr_1_intersection_x = (c_corner_edge_1 - c_right_edge)/(slope_right_edge - slope_corner_edge_1)
    cntr_1_intersection_y = slope_right_edge*cntr_1_intersection_x + c_right_edge

#find intersection points with both lines
cntr_2_intersection_x = (c_corner_edge_2 - c_left_edge)/(slope_left_edge - slope_corner_edge_2)
cntr_2_intersection_y = slope_right_edge*cntr_2_intersection_x + c_right_edge

if(not(sameSideCheck([c[cornerIndex,0,0],c[cornerIndex,0,1]],[c[0,0,0],c[0,0,1]],[cntr_2_intersection_x,cntr_2_intersection_y]))):  #use dot product to check if (x_left_corner_edge_1, y_left_corner_edge_1) is the right intersection point
    cntr_2_intersection_x = (c_corner_edge_2 - c_right_edge)/(slope_right_edge - slope_corner_edge_2)
    cntr_2_intersection_y = slope_right_edge*cntr_2_intersection_x + c_right_edge

C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_18256\1852178204.py:18: RuntimeWarning: divide by zero encountered in int_scalars
  slope_corner_edge_1 = (c[-1,0,1] - c[cornerIndex,0,1])/(c[-1,0,0] - c[cornerIndex,0,0])
C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_18256\1852178204.py:31: RuntimeWarning: invalid value encountered in double_scalars
  cntr_1_intersection_x = (c_corner_edge_1 - c_left_edge)/(slope_left_edge - slope_corner_edge_1)
C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_18256\1852178204.py:36: RuntimeWarning: invalid value encountered in double_scalars
  cntr_1_intersection_x = (c_corner_edge_1 - c_right_edge)/(slope_right_edge - slope_corner_edge_1)


In [ ]:
mountingPoint = mountingPoints[0]

slope_farthest_edge = (quadPoints[3][1]-quadPoints[2][1])/(quadPoints[3][0]-quadPoints[2][0]) #slope of farthest edge line

c2 = quadPoints[3][1] - slope_farthest_edge*quadPoints[3][0] #constant of farthest edge line

slope = (mountingPoint[1] - c[cornerIndex,0,1])/(mountingPoint[0] - c[cornerIndex,0,0])
#find the intersection point b/w the farthest edge line and the line joining cornerPoint and mountingPoint
#points are selected such that quadPoints[3] and quadPoints[4] represent the points for the farthest edge


#finding intersection point of both lines associated w/ slope
#x_intercept = (c2-c1)/(m1-m2)
#y_intercept = m1*x_intercept + c1

c1 = mountingPoint[1] - slope*mountingPoint[0] #constant of line joining mounting point and corner = y1 - mx1

#have obtained one point for the blacked out region
x_intercept = (c2-c1)/(slope - slope_farthest_edge)
y_intercept = slope*x_intercept + c1